In [ ]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader, random_split
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

import time
import os
os.environ['KMP_DUPLICATE_?LIB_OK']='True'

from dataset import *
from transforms import *
from criteria import *

from gammanet import GammaNet
from pytorch_unet.unet import UNet

In [ ]:
# batch options 

rounds = 5
epochs = 2

save_model = True
save_period = 1
save_loss = True

experiment_code = os.environ["EXP_CODE"]
save_root = "{}/".format(experiment_code)
log_dir = "{}/{}/".format(os.environ["CODE_ROOT"], os.environ["LOG_SUBDIR"])

model_file_template = save_root + "model_r{}_e{}.pkl"
loss_file_path = save_root + "loss_r{}.pkl"
loss_png_path = save_root + "loss_r{}.png"

status_file = save_root + "live.out"
log_file = log_dir + "{}.log".format(experiment_code)

# Datasets

In [ ]:
# ACDC

# NOTE: data stays in CPU until right before forward pass (i.e. no need for memory pinning, etc)

x_transform = torchvision.transforms.Compose([GaussianSmooth(3,1),
                                              CLAHE(clipLimit=2.0, tileGridSize=(8,8)),
                                              MinMax(),
                                              PadOrCenterCrop(size=(176,176)),
                                              ToTensor(make_CHW=True, out_type=float)])
#                                               TimeSeriesToTensor(make_TCHW=True, out_type=float)])
y_transform = torchvision.transforms.Compose([PadOrCenterCrop(size=(176,176)),
                                              ToTensor(make_CHW=False, out_type=int),
                                              SelectClass(3)])

# data_dir = "ACDC_split_ts4/training_8-2/"
data_dir = "ACDC_split/training_8-2/"
ds_train = SimpleDataset(data_dir + "x_train.txt", data_dir + "y_train.txt",
                         x_transform=x_transform, y_transform=y_transform, use_cache=True)
ds_val = SimpleDataset(data_dir + "x_val.txt", data_dir + "y_val.txt",
                         x_transform=x_transform, y_transform=y_transform, use_cache=True)

dl_train = DataLoader(ds_train, batch_size=2, shuffle=True, num_workers=0, pin_memory=True)
dl_val = DataLoader(ds_val, batch_size=16, shuffle=True, num_workers=0, pin_memory=True)

In [ ]:
# sunnybrook

x_transform = torchvision.transforms.Compose([GaussianSmooth(3,1),
                                              MinMax(),
                                              PadOrCenterCrop(size=(176,176)),
                                              ToTensor(make_CHW=True, input_format="HWC", out_type=float)])
y_transform = torchvision.transforms.Compose([PadOrCenterCrop(size=(176,176)),
                                              ToTensor(make_CHW=False, out_type=int)])
sb_ds_val = SimpleDataset("sunnybrook_180/val/images.npy", "sunnybrook_180/val/masks.npy", 
                       x_transform=x_transform, y_transform=y_transform, use_cache=False)
sb_dl_val = DataLoader(sb_ds_val, batch_size=8, shuffle=True, num_workers=0, pin_memory=True)

# Training model

In [ ]:
# model setup 

def init_model():
    
    gammanet_config = GammaNet._get_default_config()
    # gammanet_config["input_timeseries"] = True
    gammanet_config["fgru_timesteps"] = 2  # necessary for time series input
    
    # initialize model
    model = nn.Sequential(
        GammaNet(gammanet_config),
        nn.ReLU(), 
        nn.BatchNorm2d(24, eps=1e-3),
        nn.Conv2d(24, 1, 5, padding=2),  # Change the expected number of output classes! 
    )
    if torch.cuda.is_available():
        model = model.cuda().float()
    else:
        model = model.double()
        
    # load model?
    load_model = False
    model_file = "models_gn_8_1-9/model_e200.pkl"
    if load_model:
        if torch.cuda.is_available():
            model.load_state_dict(torch.load(model_file))
        else:
            model.load_state_dict(torch.load(model_file, map_location=torch.device('cpu')))
    
    # parallel
    model = nn.DataParallel(model)

    return model

In [ ]:
from datetime import datetime, timedelta

if not os.path.exists(save_root):
    os.makedirs(save_root)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

for r in range(rounds):
    
    torch.manual_seed(42+r)  # different seed each time

    # init model
    model = init_model()
            
    # criterion
    criterion = nn.BCEWithLogitsLoss()
    criterion = criterion.cuda() if torch.cuda.is_available() else criterion
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    
    # initialize training loop
    train_loss = []
    val_loss = []
    val_dice = []
    sb_dice = []
    
    fig, ax = plt.subplots(1, 1)
    ax.set_ylim(0,1)
    hl_train, = ax.plot([], [])
    hl_val, = ax.plot([], [])
    hl_val_dice, = ax.plot([], [])
    hl_sb_dice, = ax.plot([], [])
    fig.canvas.draw()

    # start training
    start_time = datetime.now()
    for epoch in range(epochs):  # loop over the dataset multiple times

        model.train()
        train_loss_epoch = []
        for i, data in enumerate(dl_train):
            inputs = data[0].cuda().float() if torch.cuda.is_available() else data[0].double()
            labels = data[1].cuda().float() if torch.cuda.is_available() else data[1].double()

            optimizer.zero_grad()
            outputs = model.forward(inputs).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss_epoch += [loss.item()]

        model.eval()
        val_loss_epoch = []
        val_dice_epoch = []
        for i, data in enumerate(dl_val):
            inputs = data[0].cuda().float() if torch.cuda.is_available() else data[0].double()
            labels = data[1].cuda().float() if torch.cuda.is_available() else data[1].double()

            with torch.no_grad():
                outputs = model.forward(inputs).squeeze()
                loss = criterion(outputs, labels)
                dice = dice_coeff(torch.sigmoid(outputs), labels)
            val_loss_epoch += [loss.item()]
            val_dice_epoch += [dice.item()]

        # compute dice for SB
        sb_dice_epoch = []
        for i, data in enumerate(sb_dl_val):
            inputs = data[0].cuda().float() if torch.cuda.is_available() else data[0].double()
            labels = data[1].cuda().float() if torch.cuda.is_available() else data[1].double()
            
            with torch.no_grad():
                outputs = model.forward(inputs).squeeze()
                dice = dice_coeff(torch.sigmoid(outputs), labels)
            sb_dice_epoch += [dice.item()]

        train_loss += [train_loss_epoch]
        val_loss += [val_loss_epoch]
        val_dice += [val_dice_epoch]
        sb_dice += [sb_dice_epoch]
                
        # update loss graph
        hl_train.set_xdata(np.append(hl_train.get_xdata(), epoch+1))
        hl_train.set_ydata(np.append(hl_train.get_ydata(), np.mean(train_loss_epoch)))
        hl_val.set_xdata(np.append(hl_val.get_xdata(), epoch+1))
        hl_val.set_ydata(np.append(hl_val.get_ydata(), np.mean(val_loss_epoch)))
        hl_val_dice.set_xdata(np.append(hl_val_dice.get_xdata(), epoch+1))
        hl_val_dice.set_ydata(np.append(hl_val_dice.get_ydata(), np.mean(val_dice_epoch)))
        hl_sb_dice.set_xdata(np.append(hl_sb_dice.get_xdata(), epoch+1))
        hl_sb_dice.set_ydata(np.append(hl_sb_dice.get_ydata(), np.mean(sb_dice_epoch)))
        ax.legend(['Train','Val','Val_Dice','SB_Dice']); 
        ax.relim(); ax.autoscale(axis='x'); fig.canvas.draw()

        # save model
        if save_model and (epoch+1) % save_period == 0:
            if isinstance(model, nn.DataParallel):
                torch.save(model.module.state_dict(), model_file_template.format(r+1, epoch+1))
            else:
                torch.save(model.state_dict(), model_file_template.format(r+1, epoch+1))

        # print epoch summary
        progress = (epoch+1) / epochs
        time_elapsed = datetime.now() - start_time
        time_to_completion = time_elapsed / progress - time_elapsed
        summary_str = "Round: {}, Epoch: {}, Val Loss: {}, Val Dice: {}, SB Dice: {}, ETA: {}\n"
        summary_str = summary_str.format(r+1, epoch+1, np.mean(val_loss_epoch),
                                         np.mean(val_dice_epoch), np.mean(sb_dice_epoch), 
                                         str(time_to_completion))
        print(summary_str)
        with open(log_file, 'a') as f:
            f.write(summary_str)

    if save_loss:
        pickle.dump((train_loss, val_loss, val_dice, sb_dice), 
                    open(loss_file_path.format(r+1), 'wb'))
        plt.savefig(loss_png_path.format(r+1))
        